In [5]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 19.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.3 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039163 sha256=c41f56b3bb18e1d97edde653ce8625643f4a35cd53ef59300ea01cf2c11b064c
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 s

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [6]:
import nltk
!pip install umap-learn
nltk.download('stopwords')
from umap import UMAP # Dimension reduction
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import os
import json
import gzip
from urllib.request import urlopen

In [8]:
def open_to_df(file):
  data = []
  with gzip.open(file) as f:
      for l in f:
          data.append(json.loads(l.strip()))

  df = pd.DataFrame.from_dict(data)
  return df

In [9]:
!wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/metaFiles2/meta_Appliances.json.gz

--2023-12-09 22:33:06--  https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/metaFiles2/meta_Appliances.json.gz
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59884788 (57M) [application/x-gzip]
Saving to: ‘meta_Appliances.json.gz’

meta_Appliances.jso 100%[===================>]  57.11M  66.9MB/s    in 0.9s    

2023-12-09 22:33:07 (66.9 MB/s) - ‘meta_Appliances.json.gz’ saved [59884788/59884788]



In [10]:
metadata = open_to_df('meta_Appliances.json.gz')
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,"[Appliances, Refrigerators, Freezers & Ice Mak...","class=""a-keyvalue prodDetTable"" role=""present...",[],,Tupperware Freezer Square Round Container Set ...,[],,Tupperware,[Each 3-pc. set includes two 7/8-cup/200 mL an...,"[>#39,745 in Appliances (See top 100)]",[],{},Appliances,,"November 19, 2008",,7301113188,[],[]
1,"[Appliances, Refrigerators, Freezers & Ice Mak...","class=""a-keyvalue prodDetTable"" role=""present...",[2 X Tupperware Pure & Fresh Unique Covered Co...,,2 X Tupperware Pure &amp; Fresh Unique Covered...,[],,Tupperware,[2 X Tupperware Pure & Fresh Unique Covered Co...,"[>#6,118 in Appliances (See top 100)]",[B004RUGHJW],{},Appliances,,"June 5, 2016",$3.62,7861850250,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,"[Appliances, Parts &amp; Accessories]",,[],,The Cigar - Moments of Pleasure,[],,The Cigar Book,[],"[>#1,861,816 in Home &amp; Kitchen (See Top 10...","[B01HCAVSLK, 1632206579]",{},Amazon Home,,,$150.26,8792559360,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,"[Appliances, Parts & Accessories]","class=""a-keyvalue prodDetTable"" role=""present...","[Multi purpost descaler, especially suited to ...",,Caraselle 2X 50G Appliance Descalene,[],,Caraselle,[],"[>#1,654,505 in Tools & Home Improvement (See ...",[],{},Tools & Home Improvement,,"December 17, 2014",.a-box-inner{background-color:#fff}#alohaBuyBo...,9792954481,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,"[Appliances, Parts & Accessories, Range Parts ...","class=""a-keyvalue prodDetTable"" role=""present...",[Full gauge and size beveled-edge; furnished w...,,EATON Wiring 39CH-SP-L Arrow Hart 1-Gang Chrom...,[],,EATON Wiring,[Returns will not be honored on this closeout ...,"[>#3,066,990 in Tools & Home Improvement (See ...",[],{},Tools & Home Improvement,,"January 16, 2007",$3.43,B00002N5EL,[],[]


In [11]:
!wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/Appliances.json.gz

--2023-12-09 22:33:22--  https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/Appliances.json.gz
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69677301 (66M) [application/x-gzip]
Saving to: ‘Appliances.json.gz’

Appliances.json.gz  100%[===================>]  66.45M  68.7MB/s    in 1.0s    

2023-12-09 22:33:23 (68.7 MB/s) - ‘Appliances.json.gz’ saved [69677301/69677301]



In [12]:
userrating = open_to_df('Appliances.json.gz')
userrating = userrating.iloc[35:, :] # removing erroneous data
userrating.head(3)

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
35,5.0,NaN,True,"03 20, 2015",A3SHVDMM83IHJ4,B00002N7IL,{'Size:': ' Pack of 1'},steve crumpler,Just what I needed for my electric range. Matc...,Matched pigtail cord and works great.,1426809600,NaN
36,5.0,NaN,True,"04 20, 2013",A2OXDRWBASV91Y,B00004SQHD,"{'Size:': ' 6 ft', 'Style:': ' (Old)'}",Harold E. Ewing,I like the fact that the wire ends have mounti...,complete package,1366416000,NaN
37,5.0,2,True,"03 16, 2013",A2KG6AWJSWILPR,B00004SQHD,"{'Size:': ' 6 ft', 'Style:': ' (Old)'}",Christopher J Park,"Needed another couple of feet with new dryer, ...",Perfect Fit,1363392000,NaN


In [13]:
completedata = pd.merge(userrating, metadata, on='asin', how = 'inner')
completedata.head(5)

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,...,feature,rank,also_view,details,main_cat,similar_item,date,price,imageURL,imageURLHighRes
0,5.0,NaN,True,"03 20, 2015",A3SHVDMM83IHJ4,B00002N7IL,{'Size:': ' Pack of 1'},steve crumpler,Just what I needed for my electric range. Matc...,Matched pigtail cord and works great.,...,"[Fits all wallplates with 2.15"" diameter cente...","[>#411,187 in Tools & Home Improvement (See to...","[B00002N7HY, B074Q2KZGV, B000FPCL2K, B000FP8LK...",{},Tools & Home Improvement,"class=""a-bordered a-horizontal-stripes a-spa...","July 7, 2004",,[],[]
1,5.0,NaN,True,"03 20, 2015",A3SHVDMM83IHJ4,B00002N7IL,{'Size:': ' Pack of 1'},steve crumpler,Just what I needed for my electric range. Matc...,Matched pigtail cord and works great.,...,"[Fits all wallplates with 2.15"" diameter cente...","[>#411,187 in Tools & Home Improvement (See to...","[B00002N7HY, B074Q2KZGV, B000FPCL2K, B000FP8LK...",{},Tools & Home Improvement,"class=""a-bordered a-horizontal-stripes a-spa...","July 7, 2004",,[],[]
2,5.0,NaN,True,"03 26, 2018",A3TIWHNJXMSIU7,B00002N7IL,{'Size:': ' Pack of 1'},Torpex,this particular type is perfect and easily ada...,this particular type is perfect and easily ada...,...,"[Fits all wallplates with 2.15"" diameter cente...","[>#411,187 in Tools & Home Improvement (See to...","[B00002N7HY, B074Q2KZGV, B000FPCL2K, B000FP8LK...",{},Tools & Home Improvement,"class=""a-bordered a-horizontal-stripes a-spa...","July 7, 2004",,[],[]
3,5.0,NaN,True,"03 26, 2018",A3TIWHNJXMSIU7,B00002N7IL,{'Size:': ' Pack of 1'},Torpex,this particular type is perfect and easily ada...,this particular type is perfect and easily ada...,...,"[Fits all wallplates with 2.15"" diameter cente...","[>#411,187 in Tools & Home Improvement (See to...","[B00002N7HY, B074Q2KZGV, B000FPCL2K, B000FP8LK...",{},Tools & Home Improvement,"class=""a-bordered a-horizontal-stripes a-spa...","July 7, 2004",,[],[]
4,5.0,NaN,True,"03 20, 2018",AVP16JFIT6LPL,B00002N7IL,{'Size:': ' Pack of 1'},RAFAEL FERNANDEZ,excellent,Five Stars,...,"[Fits all wallplates with 2.15"" diameter cente...","[>#411,187 in Tools & Home Improvement (See to...","[B00002N7HY, B074Q2KZGV, B000FPCL2K, B000FP8LK...",{},Tools & Home Improvement,"class=""a-bordered a-horizontal-stripes a-spa...","July 7, 2004",,[],[]


In [14]:
missing_values = completedata['reviewText'].isnull().sum()
print("Number of missing values in 'reviewText':", missing_values)
completedata['reviewText'].fillna('', inplace=True)

Number of missing values in 'reviewText': 325


In [ ]:
data = completedata.copy()

In [ ]:
completedata = data.copy()

In [16]:
completedata.shape

(615747, 30)

In [17]:
completedata = completedata.iloc[:20000]

In [ ]:
data.value_counts(['title'])

title                                                                                                                                      
Samsung Genuine DA29-00020B Refrigerator Water Filter, 3 Pack                                                                                  7404
General Electric MWF Refrigerator Water Filter                                                                                                 6510
 Gardus RLE202 LintEater Rotary Dryer Vent Cleaning System                                                                                     4048
GE MWF SmartWater Compatible Water Filter Cartridge - Refrigerator                                                                             4045
Broan 413004 ADA Capable Non-Ducted Under-Cabinet Range Hood, 30-Inch, Stainless Steel                                                         3056
                                                                                                                        

In [ ]:
completedata = completedata.iloc[:10000, :]

In [18]:
# NLTK English stopwords
stopwords = nltk.corpus.stopwords.words('english')

In [19]:
len(stopwords)

179

In [42]:
amazon_rel = ['amazon', 'buy', 'deliver', 'pack', 'also', 'product',
'filter'
'price',
'works',
'bulb',
'freezer',
'dryer',
'stove',
'covers',
'burner',
'humidifier',
'stars',
'one',
'two',
'three',
'four',
'five',
'star',
'washer',
'appliance']

In [43]:
stopwords.extend(amazon_rel)
print(f'There are {len(stopwords)} stopwords.')

There are 208 stopwords.


In [44]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=15,
                  n_components=5,
                  min_dist=0.0,
                  metric='cosine',
                  random_state=100)

# Count vectorizer
vectorizer_model = CountVectorizer(stop_words=stopwords)

In [26]:
missing_values = completedata['reviewText'].isnull().sum()
print("Number of missing values in 'reviewText':", missing_values)
completedata['reviewText'].fillna('', inplace=True)

Number of missing values in 'reviewText': 0


In [27]:
topic_model = BERTopic(umap_model=umap_model,
                       vectorizer_model=vectorizer_model,
                       min_topic_size=200,
                       top_n_words=10,
                       language="multilingual",
                       calculate_probabilities=True)

topics = topic_model.fit_transform(completedata['reviewText'])
topic_model.get_topic_info()

.gitattributes:   0%|          | 0.00/968 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

unigram.json:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,6006,-1_works_great_good_product,"[works, great, good, product, one, filter, use...","[Works great, Works great., This is the washer..."
1,0,2277,0_product_price_great_good,"[product, price, great, good, works, easy, fas...","[Great product. Great price., Great product. G..."
2,1,1928,1_filter_water_filters_ge,"[filter, water, filters, ge, refrigerator, one...",[This water filter is great. This is the first...
3,2,1644,2_stove_covers_burner_kitchen,"[stove, covers, burner, kitchen, burners, look...",[These Stove Burner Covers are beautiful. They...
4,3,1543,3_vent_dryer_air_lint,"[vent, dryer, air, lint, cold, house, plastic,...","[Update 2/12/14. After 1 yr+ operation, I hav..."
5,4,1283,4_freezer_fridge_bulb_light,"[freezer, fridge, bulb, light, refrigerator, b...",[These are great lamps for your fridge. Probab...
6,5,1211,5_humidifier_product_use_water,"[humidifier, product, use, water, humidifiers,...","[Great product for your humidifier., Great pro..."
7,6,824,6_humidifier_filter_filters_replacement,"[humidifier, filter, filters, replacement, wat...",[I only purchased one replacement filter becau...
8,7,807,7_cord_dryer_needed_cable,"[cord, dryer, needed, cable, longer, washer, w...",[We were installing a stacked washer and dryer...
9,8,642,8_works_perfectly_fit_well,"[works, perfectly, fit, well, worked, great, f...","[Fits and works perfectly, Fits and works perf..."


In [28]:
from collections import defaultdict
topic_info_per_title = defaultdict()
unique_titles = completedata['title'].unique()
min_samples_for_clustering = 10

for title in unique_titles:
    title_data = completedata[completedata['title'] == title]
    if len(title_data) >= min_samples_for_clustering:
        try:
            topic_model = BERTopic(umap_model=umap_model,
                                   vectorizer_model=vectorizer_model,
                                   min_topic_size=200,
                                   top_n_words=10,
                                   language="multilingual",
                                   calculate_probabilities=True)
            topics = topic_model.fit_transform(title_data['reviewText'])
            topic_info_per_title[title] = topic_model.get_topic_info()
        except Exception as e:
            print(f"An error occurred for title '{title}': {str(e)}. Skipped.")
    else:
        print(f"Not enough data points for title '{title}'. Skipped.")


An error occurred for title 'Leviton 5206 50 Amp, 125/250 Volt, NEMA 10-50R, 3P, 3W, Flush Mounting Receptacle, Straight Blade, Industrial Grade, Non-Grounding, Side Wired, Steel Strap, Black': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Coleman Cable 09045 5-Foot Range Cord': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Dundas Jafine MLFVK48E Semi-Rigid Aluminum Dryer Vent Kit': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Protec DynaFilter Humidifier Cartridge, Air Cleaning, 3 ct.': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Sanyo BC1206 Kegerator Beer Cooler': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'HONEYWELL HAC-504V1 Humidifier Filter Pad': k must be less than or equal to the number of training poi

In [29]:
i = 1
for title, topic_info in topic_info_per_title.items():
    print(i)
    print(f"Title: {title}")
    print("Topic Information:")
    print(topic_info)
    print("\n")
    i = i+1

1
Title: Dundas Jafine CHK100ZW CHK100ZW6 Vents, 4-Inch, White
Topic Information:
   Topic  Count                     Name  \
0     -1    522  -1_air_heat_dryer_house   

                                      Representation  \
0  [air, heat, dryer, house, lint, works, great, ...   

                                 Representative_Docs  
0  [This item does what it's intended for, and do...  


2
Title: Holmes &quot;C&quot; Humidifier Filter, HWF65PDQ-U
Topic Information:
   Topic  Count                                 Name  \
0     -1    500  -1_filter_humidifier_filters_holmes   

                                      Representation  \
0  [filter, humidifier, filters, holmes, works, o...   

                                 Representative_Docs  
0  [Works great to replace the original filter fo...  


3
Title: Holmes &quot;A&quot; Humidifier Filter, HWF62
Topic Information:
   Topic  Count                              Name  \
0     -1   1530  -1_filter_humidifier_works_great   

      

In [ ]:
unique_titles

In [ ]:
len(completedata['title'].unique())

In [ ]:
completedata.shape

In [33]:
completedata.columns

Index(['overall', 'vote', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'image', 'category', 'tech1', 'description', 'fit', 'title', 'also_buy',
       'tech2', 'brand', 'feature', 'rank', 'also_view', 'details', 'main_cat',
       'similar_item', 'date', 'price', 'imageURL', 'imageURLHighRes'],
      dtype='object')

In [45]:
from collections import defaultdict
topic_info_per_title = defaultdict()
unique_titles = completedata['brand'].unique()
min_samples_for_clustering = 2  # Adjust as needed
for title in unique_titles:
    title_data = completedata[completedata['brand'] == title]
    if len(title_data) >= min_samples_for_clustering:
        try:
            topic_model = BERTopic(umap_model=umap_model,
                                   vectorizer_model=vectorizer_model,
                                   min_topic_size=200,
                                   top_n_words=10,
                                   language="multilingual",
                                   calculate_probabilities=True)
            topics = topic_model.fit_transform(title_data['reviewText'])
            topic_info_per_title[title] = topic_model.get_topic_info()
        except Exception as e:
            print(f"An error occurred for title '{title}': {str(e)}. Skipped.")
    else:
        print(f"Not enough data points for title '{title}'. Skipped.")


An error occurred for title 'Leviton': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Coleman Cable': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Pro Tec': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Sanyo': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Carol Wright Gifts': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Watts': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Deflect-O': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'D&amp;S Specialty Products': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Air King': k must be less than or equal to the nu

In [46]:
i = 1
for title, topic_info in topic_info_per_title.items():
    print(i)
    print(f"Title: {title}")
    print("Topic Information:")
    print(topic_info)
    print("\n")
    i = i+1

1
Title: Dundas Jafine
Topic Information:
   Topic  Count                    Name  \
0     -1    606  -1_air_heat_house_vent   

                                      Representation  \
0  [air, heat, house, vent, lint, great, screen, ...   

                                 Representative_Docs  
0  [This item does what it's intended for, and do...  


2
Title: Holmes
Topic Information:
   Topic  Count                           Name  \
0     -1    933      -1_great_good_fit_perfect   
1      0   1485     0_filter_filters_water_use   
2      1    474     1_price_great_good_arrived   
3      2    312  2_holmes_filter_filters_water   
4      3    240        3_job_fine_fit_expected   

                                      Representation  \
0  [great, good, fit, perfect, smell, filter, nee...   
1  [filter, filters, water, use, good, well, last...   
2  [price, great, good, arrived, fast, find, ship...   
3  [holmes, filter, filters, water, replacement, ...   
4  [job, fine, fit, expected, 

In [39]:
topic_model = BERTopic(umap_model=umap_model,
                       vectorizer_model=vectorizer_model,
                       min_topic_size=200,
                       top_n_words=10,
                       language="multilingual",
                       calculate_probabilities=True)

topics = topic_model.fit_transform(completedata['summary'])
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,5785,-1_great_works_filter_good,"[great, works, filter, good, perfect, humidifi...","[Works great., works great, Works great]"
1,0,4761,0_product_good_price_great,"[product, good, price, great, works, quality, ...","[Great Product at a Great Price, Good product ..."
2,1,4518,1_five_stars_jingle_yr,"[five, stars, jingle, yr, tune, bells, wind, o...","[Five Stars, Five Stars, Five Stars]"
3,2,1397,2_vent_dryer_humidifier_cold,"[vent, dryer, humidifier, cold, humidity, heat...","[Great dryer vent, Great Dryer Vent, Great dry..."
4,3,968,3_four_stars_wonderful_sones,"[four, stars, wonderful, sones, vent, , , , , ]","[Four Stars, Four Stars, Four Stars]"
5,4,508,4_filter_filters_replacement_good,"[filter, filters, replacement, good, last, ge,...","[It's a filter, It's a filter, It's a filter]"
6,5,459,5_fridge_bulb_refrigerator_freezer,"[fridge, bulb, refrigerator, freezer, light, g...","[great bulb for the fridge, Great light for a ..."
7,6,352,6_three_stars_third_star,"[three, stars, third, star, , , , , , ]","[Three Stars, Three Stars, Three Stars]"
8,7,290,7_cord_dryer_wire_cable,"[cord, dryer, wire, cable, 10, prong, power, p...","[Dryer cord, Dryer Cord, Dryer Cord]"
9,8,286,8_water_filter_treatment_taste,"[water, filter, treatment, taste, clean, tasti...","[water filter, Water Filter, Water Filter]"


In [40]:
topic_info_per_title = defaultdict()
unique_titles = completedata['brand'].unique()
min_samples_for_clustering = 2
for title in unique_titles:
    title_data = completedata[completedata['brand'] == title]
    if len(title_data) >= min_samples_for_clustering:
        try:
            topic_model = BERTopic(umap_model=umap_model,
                                   vectorizer_model=vectorizer_model,
                                   min_topic_size=200,
                                   top_n_words=10,
                                   language="multilingual",
                                   calculate_probabilities=True)
            topics = topic_model.fit_transform(title_data['summary'])
            topic_info_per_title[title] = topic_model.get_topic_info()
        except Exception as e:
            print(f"An error occurred for title '{title}': {str(e)}. Skipped.")
    else:
        print(f"Not enough data points for title '{title}'. Skipped.")


An error occurred for title 'Leviton': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Coleman Cable': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Pro Tec': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Sanyo': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Carol Wright Gifts': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Watts': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Deflect-O': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'D&amp;S Specialty Products': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Air King': k must be less than or equal to the nu

In [41]:
i = 1
for title, topic_info in topic_info_per_title.items():
    print(i)
    print(f"Title: {title}")
    print("Topic Information:")
    print(topic_info)
    print("\n")
    i = i+1

1
Title: Dundas Jafine
Topic Information:
   Topic  Count                       Name  \
0     -1    606  -1_stars_works_heat_great   

                                      Representation  \
0  [stars, works, heat, great, five, good, easy, ...   

                    Representative_Docs  
0  [Five Stars, Five Stars, Five Stars]  


2
Title: Holmes
Topic Information:
   Topic  Count                               Name  \
0     -1    456  -1_holmes_filter_three_humidifier   
1      0   1360         0_good_product_works_great   
2      1    822                     1_five_stars__   
3      2    390  2_filter_filters_good_replacement   
4      3    212    3_humidifier_filter_sunbeam_air   
5      4    204                     4_four_stars__   

                                      Representation  \
0  [holmes, filter, three, humidifier, star, repl...   
1  [good, product, works, great, price, perfect, ...   
2                      [five, stars, , , , , , , , ]   
3  [filter, filters, good, r

In [47]:
topic_info_per_title = defaultdict()
unique_titles = completedata['title'].unique()
min_samples_for_clustering = 2
for title in unique_titles:
    title_data = completedata[completedata['title'] == title]
    if len(title_data) >= min_samples_for_clustering:
        try:
            topic_model = BERTopic(umap_model=umap_model,
                                   vectorizer_model=vectorizer_model,
                                   min_topic_size=200,
                                   top_n_words=10,
                                   language="multilingual",
                                   calculate_probabilities=True)
            topics = topic_model.fit_transform(title_data['reviewText'])
            topic_info_per_title[title] = topic_model.get_topic_info()
        except Exception as e:
            print(f"An error occurred for title '{title}': {str(e)}. Skipped.")
    else:
        print(f"Not enough data points for title '{title}'. Skipped.")


An error occurred for title 'Leviton 5206 50 Amp, 125/250 Volt, NEMA 10-50R, 3P, 3W, Flush Mounting Receptacle, Straight Blade, Industrial Grade, Non-Grounding, Side Wired, Steel Strap, Black': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Coleman Cable 09045 5-Foot Range Cord': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Dundas Jafine MLFVK48E Semi-Rigid Aluminum Dryer Vent Kit': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Protec DynaFilter Humidifier Cartridge, Air Cleaning, 3 ct.': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'Sanyo BC1206 Kegerator Beer Cooler': k must be less than or equal to the number of training points. Skipped.
An error occurred for title 'HONEYWELL HAC-504V1 Humidifier Filter Pad': k must be less than or equal to the number of training poi

In [48]:
topic_info_per_title

defaultdict(None,
            {'Dundas Jafine CHK100ZW CHK100ZW6 Vents, 4-Inch, White':    Topic  Count                    Name  \
             0     -1    522  -1_air_heat_house_lint   
             
                                                   Representation  \
             0  [air, heat, house, lint, great, screen, vent, ...   
             
                                              Representative_Docs  
             0  [This item does what it's intended for, and do...  ,
             'Holmes &quot;C&quot; Humidifier Filter, HWF65PDQ-U':    Topic  Count                            Name  \
             0     -1    500  -1_filter_filters_holmes_water   
             
                                                   Representation  \
             0  [filter, filters, holmes, water, last, great, ...   
             
                                              Representative_Docs  
             0  [With this brand new filter the filter status ...  ,
             'Holmes &quo